Complete the Individual Assignment
We’ve learned how the concept of cross-validation can be applied during model training to assess the performance of a model when it is applied to previously unseen data.

For this assignment, your primary task is to construct a pair of cross-validated linear regression models that predict the weight of a vehicle. The cross-validated performance of the two models is then compared to identify the preferred model.

There are two deliverables for this assignment:

A Jupyter Notebook containing all of your narrative + Python code.

A short video presentation of your work. Note that you do not need to appear on camera. Your second deliverable for this assignment(10 Points)is a short (approx. 5 minute) video presentationof your work. Your presentation should include a brief overview of your EDA findings, a high-level explanation of your regression models, and a summary of your cross validation findings + which model you prefer. 

details: https://yu.instructure.com/courses/50398/assignments/191947?module_item_id=704947

The data set you will be using is sourced from the UC Irvine machine learning archive: https://archive.ics.uci.edu/ml/datasets/Automobile

#### 1)Introduction(5 Points):  Summarize the problem + explain the steps you plan to take to address the problem
#### 2)Exploratory Data Analysis(35Points): Explain + present your EDA work including any conclusions you draw from your analysis including any preliminary predictive inferences. This section should include any Python code usedfor the EDA. 
#### 3)Regression Model Construction & Evaluation(45Points): Explain + present your linear regression models. This section should include an explanation of how you decided upon the explanatory variables to include in the models, how you implemented your K-fold cross validation, and any Python code used for construction + evaluation of the regression models.
#### 4)Conclusions(5 Points)

#### 1)Load the provided M2_Data.csv file to your DAV 6150 Github Repository. 


In [1]:
# load the pandas library
import pandas as pd

# load the train_test_split function from the sklearn.model_selection module
from sklearn.model_selection import train_test_split

# start by reading a set of sample data from github. This data set contains information related to flights
# departing from the two major airports in Houston, Texas
filename = "https://raw.githubusercontent.com/jtopor/DAV-5400/master/Project1/hflights.csv"
df = pd.read_csv(filename)
df.head()

,Year,Month,DayofMonth,DayOfWeek,DepTime,ArrTime,UniqueCarrier,FlightNum,TailNum,ActualElapsedTime,...,ArrDelay,DepDelay,Origin,Dest,Distance,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted
0,2011,1,1,6,1400.0,1500.0,AA,428,N576AA,60.0,...,-10.0,0.0,IAH,DFW,224,7.0,13.0,0,NaN,0
1,2011,1,2,7,1401.0,1501.0,AA,428,N557AA,60.0,...,-9.0,1.0,IAH,DFW,224,6.0,9.0,0,NaN,0
2,2011,1,3,1,1352.0,1502.0,AA,428,N541AA,70.0,...,-8.0,-8.0,IAH,DFW,224,5.0,17.0,0,NaN,0
3,2011,1,4,2,1403.0,1513.0,AA,428,N403AA,70.0,...,3.0,3.0,IAH,DFW,224,9.0,22.0,0,NaN,0
4,2011,1,5,3,1405.0,1507.0,AA,428,N492AA,62.0,...,-3.0,5.0,IAH,DFW,224,9.0,9.0,0,NaN,0


#### 2)Then, using a Jupyter Notebook, read the data set from your Github repository and load it into a Pandas dataframe. 



####  3)Using your Python skills, perform some basic exploratorydata analysis (EDA) to ensure you understand the nature of each of the variables (including the response variable). Your EDA writeup should include any insights you are able to derive from your statistical analysis of the attributes and the accompanying exploratory graphics you create (e.g., bar plots, box plots, histograms, line plots, etc.). You should also try to identify some preliminary predictive inferences, e.g., do any of the explanatory variables appear to be relatively more “predictive” of the response variable? There are a variety of ways you can potentially identify such relationships between the explanatory variables and the response variable. It is up to you as the data science practitioner to decide how you go about your EDA, including selecting appropriate statistical metrics to be calculated + which types of exploratory graphics to make use of. Your goal should be to provide an EDA that is thorough and succinct without it being so detailed that a reader will lose interest in it.

#### 4)Using your Python skills, construct at least two different linear regression modelsthat predict curb-weightbased on the provided explanatory variables and evaluatethem using K-fold cross validation. Each of your modelsmust include at leastthree (3)explanatory variables. The explanatory variables you choose to include within each of your models should reflect some of the knowledge you have gained via your EDA work. The value(s) you choose for ‘K’ for your cross validation are up to you as a datasciencepractitioner to select. After executing your K-fold cross validation for each of you models, select your preferred model based on the average R^2scores you derived viathe K-fold process.